In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import psycopg2
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import polars as pl
from sqlalchemy import create_engine, text
from tqdm import tqdm
from geopy.distance import geodesic
from pymongo import MongoClient
from shapely.geometry import Point, LineString


client = MongoClient('localhost', 27017)
db = client['vic_db']

# Define database connection parameters
database_connection = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'vic_db',
}

# A psycopg2 connection and cursor
conn = psycopg2.connect(user=database_connection['username'],
                        password=database_connection['password'],
                        host=database_connection['host'],
                        port=database_connection['port'],
                        database=database_connection['database'])
conn.autocommit = True
cursor = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql://%(username)s:%(password)s@%(host)s/%(database)s' % database_connection, isolation_level="AUTOCOMMIT")
conn_alchemy = engine.connect()

In [2]:
# sql = f"""
# SELECT ufi, ezi_road_name_label, direction_code, road_length_meters, geom
# FROM vmtrans.tr_road_all
# """

# cursor.execute("SELECT ufi, ezi_road_name_label, direction_code, road_length_meters, geom FROM vmtrans.tr_road_all")
# roads = cursor.fetchall()
# roads_df = pd.DataFrame(roads, columns=['ufi', 'ezi_road_name_label', 'direction_code', 'road_length_meters', 'geom'])

sql = f"""
SELECT ufi, ezi_road_name_label, direction_code, road_length_meters, ST_AsText(geom) as geom
FROM vmtrans.tr_road_all
"""
sql = f"""
SELECT 
    ufi,
    ezi_road_name_label,
    direction_code,
    road_length_meters,
    STRING_TO_ARRAY(
        REGEXP_REPLACE(ST_AsText(geom), '^MULTILINESTRING\\(\\(|\\)\\)$', '', 'g'),
        ','
    ) AS geom_points
FROM 
    vmtrans.tr_road_all;
"""
cursor.execute(sql)
roads = cursor.fetchall()


In [3]:
segments_list = []
for road in tqdm(roads):
    ufi = road[0]
    line = road[-1]
    for segment in zip(line[:-1], line[1:]):
        point_start = segment[0].split(' ')
        point_end = segment[1].split(' ')
        segments_list.append([ufi, point_start[0], point_start[1], point_end[0], point_end[1]])

100%|██████████| 1235086/1235086 [04:48<00:00, 4284.46it/s]  


In [4]:
# Save segments_list to csv file
segments_df = pl.DataFrame(segments_list, schema=['ufi', 'start_lon', 'start_lat', 'end_lon', 'end_lat'])
segments_df.write_csv('segments.csv')

In [5]:
# Insert segments_list to a new table named segments
sql = f"""
DROP TABLE IF EXISTS vmtrans.segments;
CREATE TABLE vmtrans.segments (
    ufi integer,
    start_lon numeric,
    start_lat numeric,
    end_lon numeric,
    end_lat numeric
);
"""
cursor.execute(sql)


In [ ]:

for segment in tqdm(segments_list):
    point_start = segment[1].split(' ')
    point_end = segment[2].split(' ')
    sql = f"""
    INSERT INTO vmtrans.segments (ufi, x1, y1, x2, y2) VALUES ({segment[0]}, {point_start[0]}, {point_start[1]}, {point_end[0]}, {point_end[1]})
    """
    cursor.execute(sql)

In [ ]:
sql = """
-- Step 1: Add the PostGIS geometry column
ALTER TABLE vmtrans.segments 
ADD COLUMN geom geometry(LineString, 7844);
-- Step 2: Populate the geometry column
UPDATE vmtrans.segments
SET geom = ST_MakeLine(ST_SetSRID(ST_MakePoint(start_lon, start_lat), 7844), ST_SetSRID(ST_MakePoint(end_lon, end_lat), 7844));

-- Step 3: Create a spatial index on the geometry column
CREATE INDEX segment_geom_idx 
ON vmtrans.segments 
USING GIST (geom);
"""
cursor.execute(sql)
# 5m

In [14]:

def find_nearest_road_line(lon, lat, limit=1):
    """
    For a given point, for each road, find the nearest point on the road, and return the road with the shortest distance between that nearest point and the given point.
    """
    sql = f"""
    SELECT 
        ufi,
        direction_code,
        ST_X(ST_ClosestPoint(geom::geometry, ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844))) AS closest_point_x, 
        ST_Y(ST_ClosestPoint(geom::geometry, ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844))) AS closest_point_y, 
        from_ufi,
        to_ufi
    FROM vmtrans.tr_road_all
    WHERE direction_code IS NOT NULL
    ORDER BY ST_Distance(ST_ClosestPoint(geom::geometry, ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844)), ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844))
    LIMIT {limit};
    """
    cursor.execute(sql)
    return cursor.fetchall()

def find_nearest_road_segment(lon, lat, limit=1):
    """
    For a given point, for each road, find the nearest point on the road, and return the road with the shortest distance between that nearest point and the given point.
    """
    sql = f"""
    SELECT 
        ufi,
        start_lon, start_lat, end_lon, end_lat,
        ST_X(ST_ClosestPoint(geom::geometry, ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844))) AS closest_point_x, 
        ST_Y(ST_ClosestPoint(geom::geometry, ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844))) AS closest_point_y
    FROM vmtrans.segments
    ORDER BY ST_Distance(ST_ClosestPoint(geom::geometry, ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844)), ST_SetSRID(ST_MakePoint({lon}, {lat}), 7844))
    LIMIT {limit};
    """
    cursor.execute(sql)
    return cursor.fetchall()

In [15]:
find_nearest_road_segment(144.10242311600007, -38.455264802999946)

[(44885258,
  Decimal('144.10242311600007'),
  Decimal('-38.455164802999946'),
  Decimal('144.10353345200008'),
  Decimal('-38.45718785299994'),
  144.10246529481194,
  -38.45524165347164)]

In [16]:
find_nearest_road_line(144.10242311600007, -38.455264802999946)

[(44885258, 'B', 144.10246529481194, -38.45524165347164, 15412870, 15412880)]

In [ ]:
roads_df = pl.DataFrame(roads, schema=['ufi', 'ezi_road_name_label', 'direction_code', 'road_length_meters', 'geom'])

In [ ]:
def extract_coords(geom: str) -> list:
    geom = geom.replace('MULTILINESTRING((', '').replace('))', '')
    coords = geom.split(',')
    coords = [c.split(' ') for c in coords]
    return coords

In [ ]:
roads_df = roads_df.with_columns(pl.col('geom').str.replace('MULTILINESTRING((', '', literal=True).str.replace('))', '', literal=True).str.split(','))
roads_df = roads_df.with_columns(pl.col('geom').list.slice(0, -1).alias('start'), pl.col('geom').list.slice(1, None).alias('end'))

In [ ]:
# Explode start and end columns
roads_df.to_dict()

In [ ]:
roads_df['geom']

In [ ]:
# Convert MultiLineString to a list of points
from shapely import MultiLineString
x : MultiLineString = gdf['geom'][0]
def gen_segments(x : MultiLineString):    
    xline = x.geoms[0].coords[:]
    xsegments = [[xline[i], xline[i+1]] for i in range(len(xline)-1)]
    return xsegments

gdf['segments'] = gdf['geom'].apply(lambda x: gen_segments(x))

In [ ]:
gdf.explode('segments')

In [ ]:
sql = """
CREATE TABLE vmtrans.segments AS
SELECT 
    ufi,
    ST_X(point_start.geom) AS x1,
    ST_Y(point_start.geom) AS y1,
    ST_X(point_end.geom) AS x2,
    ST_Y(point_end.geom) AS y2
FROM 
    vmtrans.tr_road_all,
    LATERAL (
        SELECT (ST_DumpPoints(ST_LineMerge(geom))).geom AS geom, 
               generate_series(1, ST_NPoints(ST_LineMerge(geom)) - 1) AS idx 
        FROM vmtrans.tr_road_all
    ) AS point_start
    JOIN LATERAL (
        SELECT (ST_DumpPoints(ST_LineMerge(geom))).geom AS geom, 
               generate_series(2, ST_NPoints(ST_LineMerge(geom))) AS idx 
        FROM vmtrans.tr_road_all
    ) AS point_end
ON 
    point_start.idx + 1 = point_end.idx;

"""

cursor.execute(sql)